<a href="https://colab.research.google.com/github/dhan16/dry-ml/blob/master/mlcc/first_steps_with_tensor_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [0]:
from __future__ import print_function

import math

from IPython.display import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

raw_data = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")
df = raw_data
df

### Data: preprocess and examine

##### Data: randomize, (drop duplicates?), preprocess features and targets

In [0]:
# data: randomize, scale, examine
def preprocess_features(df):
  selected_features = df[
    ["latitude",
     "longitude",
     "housing_median_age",
     "total_rooms",
     "total_bedrooms",
     "population",
     "households",
     "median_income"]]
  processed_features = selected_features.copy()
  # dfnthetic feature.
  processed_features["rooms_per_person"] = (df["total_rooms"] / df["population"])
  return processed_features

def preprocess_targets(df):
  output_targets = pd.DataFrame()
  output_targets["median_house_value"] = (df["median_house_value"] / 1000.0)
  return output_targets

# drop duplicates?
df = df.reindex(np.random.permutation(df.index))

##### Data: examine, visualize

In [0]:
training_examples = preprocess_features(df.head(12000))
training_targets = preprocess_targets(df.head(12000))

validation_examples = preprocess_features(df.tail(5000))
validation_targets = preprocess_targets(df.tail(5000))

display(training_examples.describe())
display(validation_examples.describe())
display(training_targets.describe())
display(validation_targets.describe())

def print_min_max_diff(type_, targets):
  max_target = targets.max()
  min_target = targets.min()
  print(type_, " min: %0.3f" % min_target, " max: %0.3f" % max_target, " diff: %0.3f" % (max_target - min_target))

print_min_max_diff("training", training_targets)
print_min_max_diff("validation", validation_targets)

In [0]:
def _subplot(ax, title, features_df, targets_df):
  ax.set_title(title)
  ax.set_autoscaley_on(False)
  ax.set_ylim([32, 43])
  ax.set_autoscalex_on(False)
  ax.set_xlim([-126, -112])
  plt.scatter(features_df["longitude"],
            features_df["latitude"],
            cmap="coolwarm",
            c=targets_df["median_house_value"] / targets_df["median_house_value"].max())

plt.figure(figsize=(13, 8))
_subplot(plt.subplot(1, 2, 1), 'Validation Data', validation_examples, validation_targets)
_subplot(plt.subplot(1, 2, 2), 'Training Data', training_examples, training_targets)
_ = plt.plot()

### Model

##### input, predict, train functions

In [0]:
def input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of multiple features.
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    # print("input_fn: batch_size:", batch_size, " shuffle:", shuffle, " num_epochs:", num_epochs, " features.shape:", features.shape, " targets.shape:", targets.shape)
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(10000)
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [0]:
def predict(regressor, input_fn):
  predictions = regressor.predict(input_fn=input_fn)
  predictions = np.array([item['predictions'][0] for item in predictions])
  return predictions

def train_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets,
    periods=10):
  """Trains a linear regression model of multiple features.
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    training_examples: A `DataFrame` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column to use as target for training.
    validation_examples: A `DataFrame` containing input features for validation.
    validation_targets: A `DataFrame` containing exactly one column to use as target for validation.
  Returns:
    A `LinearRegressor` object trained on the training data.
  """

  steps_per_period = steps / periods
  features=[c for c in training_examples]
  targets=[c for c in training_targets]
  # Create a linear regressor object.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  optimizer = tf.contrib.estimator.clip_gradients_by_norm(optimizer, 5.0)
  regressor = tf.estimator.LinearRegressor(
      feature_columns=[tf.feature_column.numeric_column(c) for c in features],
      optimizer=optimizer
  )
  # 1. Create input functions.
  training_input_fn = lambda:input_fn(training_examples, training_targets, batch_size=batch_size)
  predict_training_input_fn = lambda: input_fn(training_examples, training_targets, num_epochs=1, shuffle=False)
  predict_validation_input_fn = lambda: input_fn(validation_examples, validation_targets, num_epochs=1, shuffle=False)
  
  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model with targets:", targets, "\nfeatures:", features)
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    regressor.train(input_fn=training_input_fn, steps=steps_per_period)
    # 2. Take a break and compute predictions.
    training_predictions = predict(regressor, predict_training_input_fn)
    validation_predictions = predict(regressor, predict_validation_input_fn)
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print("  period %02d, RMSE training: %0.2f, validation: %0.2f" % (period, training_root_mean_squared_error, validation_root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  return regressor

##### train with hyper parameters

In [0]:
learning_rate=0.00002
steps=500
batch_size=1

regressor=None
def do_train():
  global regressor
  regressor = train_model(
    learning_rate=learning_rate,
    steps=steps,
    batch_size=batch_size,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets,
    periods=1)

import timeit
timeit.timeit(do_train, number=1)

### Test model on test data

In [0]:
# test_data = df.sample(n=300)
test_data = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv", sep=",")
test_examples = preprocess_features(test_data)
test_input_fn = tf.estimator.inputs.pandas_input_fn(test_examples, shuffle=False)
predictions = predict(regressor, test_input_fn)
test_targets = preprocess_targets(test_data)
test_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(predictions, test_targets))

print("RMSE test: %0.2f" % test_root_mean_squared_error)